<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/pi05-test/third_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sub Task Test

Check if a model creates coherent words.

In [ ]:
model_id = "NONHUMAN-RESEARCH/pi05_ki_training_fruits"
# model_id = "lerobot/pi05_base"
# model_id = "lerobot/pi05_libero"

In [4]:
from huggingface_hub import login
login()

In [1]:
# Clone

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2098, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 2098 (delta 196), reused 195 (delta 110), pack-reused 1794 (from 2)
Receiving objects: 100% (2098/2098), 7.82 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (1339/1339), done.


In [2]:
%cd XHUMAN

/content/XHUMAN


We have several problem trying to use the transformers library, this requires a more careful study

In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 27.24s
Prepared 46 packages in 21.15s
Uninstalled 15 packages in 967ms
Installed 46 packages in 219ms
 + async-lru==2.0.5
 + av==15.1.0
 + comm==0.2.3
 + deepdiff==8.6.1
 - diffusers==0.36.0
 + diffusers==0.35.2
 + draccus==0.10.0
 + evdev==1.9.2
 + faker==40.1.2
 + feetech-servo-sdk==1.0.0
 - huggingface-hub==0.36.0
 + huggingface-hub==0.35.3
 + inquirerpy==0.3.4
 - ipykernel==6.17.1
 + ipykernel==7.1.0
 + jedi==0.19.2
 + json5==0.13.0
 + jsonlines==4.0.0
 + jupyter==1.1.1
 - jupyter-client==7.4.9
 + jupyter-client==8.8.0
 + jupyter-lsp==2.3.0
 + jupyterlab==4.5.2
 + jupyterlab-server==2.28.0
 + lerobot==0.4.3 (from git+https://github.com/huggingface/lerobot.git@46e19ae579f80ce66211afafd1c3c649c569131f)
 + mergedeep==1.3.4
 + mypy-extensions==1.1.0
 - notebook==6.5.7
 + notebook==7.5.2
 - nvidia-cudnn-cu12==9.10.2.21
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cusparselt-cu12==0.7.1
 + nvidia-cusparselt-cu12==0.6.3
 - nvi

# Take the N frame from a dataset

In [ ]:
import torch
from lerobot.datasets.lerobot_dataset import LeRobotDataset

DS_ID = "NONHUMAN-RESEARCH/TEST_RECORD_ANNOTATIONS"
dataset = LeRobotDataset(DS_ID)

In [ ]:
from xhuman.policies.pi05.modeling_pi_05 import PI05Policy

policy = PI05Policy.from_pretrained(model_id)

In [ ]:
from xhuman.policies.pi05.processor_pi05 import make_pi05_pre_post_processors_ki

pre, _ = make_pi05_pre_post_processors_ki(
    config=policy.config,
    ds_meta=dataset.meta
)

**Single batch**

In [ ]:
N = 0
batch = dataset[N]
batch.keys()

In [ ]:
batch["task"]

In [ ]:
prompt = "describe the scene"
def change_task(batch):
    batch["task"] = prompt
    return batch

batch = change_task(batch)

In [12]:
device = "cuda"
# policy.model.eval()
policy.model.to(device)

In [41]:
from lerobot.utils.constants import OBS_LANGUAGE_TOKENS, OBS_LANGUAGE_ATTENTION_MASK

processed_images, img_masks = policy._preprocess_images(batch)
tokens = batch[f"{OBS_LANGUAGE_TOKENS}"]
masks = batch[f"{OBS_LANGUAGE_ATTENTION_MASK}"]
max_decoding_steps = 100
temperature = 0.1
PALIGEMMA_EOS_TOKEN = 1

In [43]:
with torch.no_grad():
    # the input was (B, H, W, C) but the model expects (B, C, H, W).
    fixed_processed_images = []
    for img in processed_images:
        # Check if shape is (B, H, W, C=3) instead of (B, C=3, H, W)
        if img.ndim == 4 and img.shape[-1] == 3 and img.shape[1] != 3:
            img = img.permute(0, 3, 1, 2)
        fixed_processed_images.append(img)

    subtask_tokenized = policy.model.sample_subtask(
        images=fixed_processed_images, # Use the fixed images
        img_masks=img_masks,
        tokens=tokens,
        masks=masks.bool(),
        max_decoding_steps=max_decoding_steps,
        eos_token_id=PALIGEMMA_EOS_TOKEN,
        temperature=temperature
    )

In [44]:
subtask = policy._detokenize_subtask(subtask_tokenized)
subtask

tensor([[3351,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]], device='cuda:0')

# Results

The model don't produce non sense words. But it just produce a word, maybe there is some bug with the EOS token. Or maybe the adaRMS thing. Before make a fine tunning we have to be sure about it.

### Fine-Tuning Example
To fine-tune the model, you need:
1. **Ground Truth Actions**: The actual movements the robot should make.
2. **Training Loop**: A standard PyTorch loop with an optimizer.

The following code demonstrates a single training step using dummy action data.

In [ ]:
import torch.optim as optim
from lerobot.utils.constants import ACTION

# 1. Setup Optimizer
# We fine-tune all parameters here, but you could also freeze specific layers
optimizer = optim.AdamW(policy.parameters(), lr=1e-5)

# 2. Prepare a Training Batch
# We start with the existing batch containing images and text
train_batch = batch.copy()

# We need to add Ground Truth Actions for the loss calculation
# Shape: (Batch_Size, Horizon, Action_Dim)
action_dim = policy.config.max_action_dim
horizon = policy.config.chunk_size
batch_size = 1

# Creating dummy actions (all zeros) for demonstration
# In reality, this would be your recorded robot data
dummy_actions = torch.zeros((batch_size, horizon, action_dim), device=device)
train_batch[ACTION] = dummy_actions

# Optional: Add mask for text loss (if you want to train the VLM part too)
# 1 = Calculate loss for this token, 0 = Ignore
# Typically you mask the prompt and only train on the generated response
train_batch["token_loss_mask"] = torch.ones_like(masks, device=device)

# 3. Run a Training Step
policy.train() # Switch to training mode
optimizer.zero_grad()

# The forward pass automatically calculates the loss
loss, loss_dict = policy(train_batch)

# Backward pass and optimization
loss.backward()
optimizer.step()

print("TRAINING STEP COMPLETE")
print(f"Total Loss: {loss.item():.4f}")
print(f"Detailed Losses: {loss_dict}")

In [ ]:
print("\n" + "="*80)
print("Testing sample_embedding...")
print("="*80)

with torch.no_grad():
    prefix_embeddings, past_key_values, prefix_pad_masks = policy.model.sample_embedding(
        processed_images,
        img_masks,
        tokens,
        masks
    )

    print(f"Prefix embeddings shape: {prefix_embeddings.shape}")
    print(f"Prefix pad masks shape: {prefix_pad_masks.shape}")
    print(f"Past key values type: {type(past_key_values)}")
    if past_key_values is not None:
        print(f"Past key values length: {len(past_key_values)}")
